This is a Neural Network to predict commitment of students with online courses.

In [67]:
import numpy as np
import pandas as pd
import csv
import matplotlib as plt

In [68]:
data = pd.read_csv("out.csv")
data.describe()

,engagement,anger,contempt,disgust,fear,happiness,neutral,sadness,surprise
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.505380,7.447691,7.527219,7.468637,7.491103,7.474113,7.477786,7.450546,7.548911
std,0.292393,4.342329,4.344747,4.327625,4.325684,4.327359,4.313112,4.343131,4.311707
min,0.000000,0.000365,0.001694,0.001612,0.000662,0.002418,0.000616,0.000678,0.000552
25%,0.300000,3.688172,3.741956,3.720413,3.747869,3.740014,3.770971,3.693497,3.828738
50%,0.500000,7.433271,7.596299,7.429996,7.552279,7.411314,7.525319,7.423179,7.546941
75%,0.800000,11.228078,11.292224,11.206452,11.198329,11.250509,11.156155,11.212856,11.273236
max,1.000000,14.998577,14.997546,14.998057,14.998981,14.999357,14.998968,14.999237,14.999866


Aqui, vamos dividir os dados para treinamento, teste e validação da rede neural.

In [85]:
train, validate, test = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

In [93]:
#dados alvo 
#target_field = data.columns[0:]
target_field = ['engagement']
features, target = data.drop(target_field, axis=1), data[target_field]
test_features, test_target = test.drop(target_field, axis=1), test[target_field]

In [99]:
#Dados para treinamento
train_features, train_target = train[:], target[:]
val_features, val_target = features[:], target[:]

In [96]:
scaled_features = {}
for each in quant_features:
    mean, std = data_train[each].mean(), data_train[each].std()
    scaled_features[each] = [mean, std]
    data_train.loc[:, each] = (data_train[each] - mean)/std
scaled_features

{'engagement': [0.5064333333333333, 0.2932284820965971],
 'anger': [7.416321554979691, 4.340221408241407],
 'contempt': [7.518373773845748, 4.357006612333243],
 'disgust': [7.485665370052796, 4.304017158161058],
 'fear': [7.4578914561883645, 4.294313512043478],
 'happiness': [7.4538733838547175, 4.349642197843544],
 'neutral': [7.480802419928315, 4.306559367335475],
 'sadness': [7.4759898366139375, 4.326874173845003],
 'surprise': [7.472926299015346, 4.317437809683168]}

In [100]:
class RedeNeural(object):
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        #definir o numero de hidden nodes
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        #pesos estatísticos
        self.weights_input_to_hidden = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                       (self.hidden_nodes, self.input_nodes))

        self.weights_hidden_to_output = np.random.normal(0.0, self.output_nodes**-0.5, 
                                       (self.output_nodes, self.hidden_nodes))
        self.lr = learning_rate
        
        #A distrubuição aqui é sigmoide
        self.activation_function = lambda x: (1 / (1 + np.exp(-x)))
        self.activation_function_prime = lambda x: (x * ( 1 - x))
        
        self.output_activation_function = lambda x: (x)
        self.output_activation_function_prime = lambda x: (1)
        
    def train(self, inputs_list, targets_list):
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        #convertendo em vetores 2D
        
        
        hidden_inputs = np.dot(self.weights_input_to_hidden, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = np.dot(self.weights_hidden_to_output, hidden_outputs)
        final_outputs = self.output_activation_function(final_inputs)
        

        error = targets - final_outputs
        
        # Output error
        output_errors = error * self.output_activation_function_prime(final_inputs)
        #erro propagado
        hidden_errors = np.dot(output_errors.T, self.weights_hidden_to_output).T
        hidden_grad = self.activation_function_prime(hidden_outputs)
        
 
        self.weights_hidden_to_output += self.lr * np.dot(output_errors, hidden_outputs.T)
        self.weights_input_to_hidden += self.lr * np.dot((hidden_errors * hidden_grad), inputs.T)
 
        
    def run(self, inputs_list):
        inputs = np.array(inputs_list, ndmin=2).T
        
       
        hidden_inputs = np.dot(self.weights_input_to_hidden, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        
        final_inputs = np.dot(self.weights_hidden_to_output, hidden_outputs) 
        final_outputs = self.output_activation_function(final_inputs)
        
        return final_outputs

In [101]:
def ErroQuadraticoMedio(y, Y):
    return np.mean((y-Y)**2)

In [102]:
import sys

epocas = 100
learning_rate = 0.05
hidden_nodes = 20
output_nodes = 1

N_i = train.shape[1]
rede = RedeNeural(N_i, hidden_nodes, output_nodes, learning_rate)

losses = {'train':[], 'validation':[]}
for e in range(epocas):
    batch = np.random.choice(train.index, size=128)
    
    for record, target in zip(train_features.ix[batch].values,
                              target[batch]['engagement']):network.train(record, target)
    
    train_loss = ErroQuadraticoMedio(rede.run(train), train_target['engagement'].values)
    val_loss = ErroQuadraticoMedio(rede.run(val_features), val_targets['engagement'].values)
    sys.stdout.write("\rProgress: " + str(100 * e/float(epochs))[:4] \
                     + "% ... Training loss: " + str(train_loss)[:5] \
                     + " ... Validation loss: " + str(val_loss)[:5])
    
    losses['train'].append(train_loss)
    losses['validation'].append(val_loss)

c:\users\ivan\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


KeyError: '[6853 9035 2253  405 5754 6613 8878 2360 1796 5772 6371 9772 6505 6043\n 8592 8378 1405 1211 9081 5480 1181 5014 7272 8941 9992 2423  937  372\n 2658  905 2940 8269 8018  211 8910 8545 4899 8923 6883 1776  598  127\n 9957 5751 7358 3381 7191 7564  858  267 6994 5156 4867 4174 7619 7698\n 3100 3374 1139 7792 6878 8179 5219 4698 5398 9337 7196 8449 8360 4739\n 9916 6325 9957 3349 5117 2789 3695 7596 5496 2854 1060 1186  325 1907\n 5713  663 2253 9238 4717 2487 7134 6906 1287 1992 5472 6875 9721 2684\n 1137 4934 2873 8842  379 7640 4636 4322 9671 4591 9271 2587 1835 1270\n 1004  720 6298 5394 1260 9396 9063  620 7184 7250 3010 4730  674 6857\n  218 6665] not in index'

In [103]:
y = data.as_matrix(columns=data.columns[0:1])
x = data.as_matrix(columns=data.columns[1:])
data.columns[0:]

c:\users\ivan\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
c:\users\ivan\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


Index(['engagement', 'anger', 'contempt', 'disgust', 'fear', 'happiness',
       'neutral', 'sadness', 'surprise'],
      dtype='object')